自动生成chat history摘要

In [4]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";
import { load } from "dotenv";
const env = await load();
const process = { env };
// 摘要model
var summaryModel = new ChatOpenAI({
  configuration: {
    baseURL: process.env.baseURL,
  },
});
var summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary retu
Current summary: {summary}
New lines of conversation: {new_lines}
New summary:
`);
var summaryChain = RunnableSequence.from([summaryPrompt, summaryModel, new StringOutputParser()]);
var newSummary = await summaryChain.invoke({
  summary: "",
  new_lines: "我18岁",
});
await summaryChain.invoke({
  summary: newSummary,
  new_lines: "我是男的",
});
// 聊天model
var chatModel = new ChatOpenAI({
  configuration: {
    baseURL: process.env.baseURL,
  },
});
var chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.
    Here is the chat history summary:     {history_summary}
    `,
  ],
  ["human", "{input}"],
]);
var summary = "";
var history = new ChatMessageHistory();
var chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => history.addUserMessage(input),
    }),
  },
  RunnablePassthrough.assign({
    history_summary: () => summary,
  }),
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      history.addAIChatMessage(input);
      const messages = await history.getMessages();
      const new_lines = getBufferString(messages);
      const newSummary = await summaryChain.invoke({
        summary,
        new_lines,
      });
      history.clear();
      summary = newSummary;
    },
  }),
]);
let res = await chatChain.invoke('我现在饿了');
console.log(res)


如果你现在感觉饿了，可以考虑吃点东西来补充能量。以下是一些常见的小吃建议：

1. **水果**：水果是一个健康的选择，比如香蕉、苹果、橙子等，它们富含维生素和纤维。
   
2. **坚果**：例如杏仁、核桃或者坚果混合物，它们富含健康脂肪和蛋白质。

3. **酸奶**：含有蛋白质和钙，可以选择低脂或者无脂酸奶。

4. **谷类食品**：例如全麦面包、饼干或者麦片，提供能量并且有助于长时间保持饱腹感。

5. **蔬菜**：例如胡萝卜、黄瓜等生吃或者配合一些低脂蘸酱，提供纤维和维生素。

6. **蛋类**：煮熟的鸡蛋或者蛋白酒炒蛋，富含优质蛋白质。

希望这些建议能帮助你找到合适的小吃来缓解饥饿感！
